In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24771
2,Huelva,Confirmados PDIA 14 días,276
3,Huelva,Tasa PDIA 14 días,"53,78334664925853"
4,Huelva,Confirmados PDIA 7 días,134
5,Huelva,Total Confirmados,24980
6,Huelva,Curados,17656
7,Huelva,Fallecidos,334


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24771.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6586.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 2714 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24771.0,134.0,276.0,513170.0,26.112205,53.783347,37.0
Huelva-Costa,15108.0,80.0,166.0,289548.0,27.629270,57.330736,23.0
Condado-Campiña,6952.0,53.0,107.0,156231.0,33.924125,68.488328,14.0
Huelva (capital),6586.0,21.0,53.0,143837.0,14.599860,36.847265,7.0
Isla Cristina,1989.0,11.0,14.0,21393.0,51.418688,65.441967,5.0
Lepe,1999.0,6.0,13.0,27880.0,21.520803,46.628407,4.0
Puebla de Guzmán,102.0,9.0,20.0,3092.0,291.073739,646.830530,4.0
San Bartolomé de la Torre,203.0,8.0,11.0,3761.0,212.709386,292.475405,3.0
Almonte,878.0,10.0,28.0,24507.0,40.804668,114.253071,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,102.0,9.0,20.0,3092.0,291.073739,646.830530,4.0
Lucena del Puerto,160.0,3.0,13.0,3261.0,91.996320,398.650721,1.0
San Silvestre de Guzmán,13.0,2.0,2.0,614.0,325.732899,325.732899,1.0
San Bartolomé de la Torre,203.0,8.0,11.0,3761.0,212.709386,292.475405,3.0
Ayamonte,1219.0,21.0,32.0,21104.0,99.507202,151.630023,1.0
Almonte,878.0,10.0,28.0,24507.0,40.804668,114.253071,3.0
Villablanca,177.0,1.0,3.0,2885.0,34.662045,103.986135,NaN
Cartaya,957.0,6.0,20.0,20083.0,29.876015,99.586715,0.0
Palma del Condado (La),981.0,3.0,10.0,10801.0,27.775206,92.584020,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,160.0,3.0,13.0,3261.0,91.996320,398.650721,1.0,0.230769
Palma del Condado (La),981.0,3.0,10.0,10801.0,27.775206,92.584020,1.0,0.300000
Cartaya,957.0,6.0,20.0,20083.0,29.876015,99.586715,0.0,0.300000
Villablanca,177.0,1.0,3.0,2885.0,34.662045,103.986135,0.0,0.333333
Almonte,878.0,10.0,28.0,24507.0,40.804668,114.253071,3.0,0.357143
Huelva (capital),6586.0,21.0,53.0,143837.0,14.599860,36.847265,7.0,0.396226
Moguer,835.0,6.0,15.0,21867.0,27.438606,68.596515,1.0,0.400000
Puebla de Guzmán,102.0,9.0,20.0,3092.0,291.073739,646.830530,4.0,0.450000
Lepe,1999.0,6.0,13.0,27880.0,21.520803,46.628407,4.0,0.461538
